In [25]:
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(ca)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import metpy.constants as constants

# 读取观测数据

In [12]:
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]/86400
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)
preGPCP_JJA = ca.detrend_dim(preGPCP_JJA, "time", deg=1, demean=False)

fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
# hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
hgtERA5_ver_JJA = ca.detrend_dim(hgtERA5_ver_JJA, "time", deg=1, demean=False)

uERA5_ver_MAM, uERA5_ver_JJA, uERA5_ver_SON, uERA5_ver_DJF = ca.mon_to_season(uERA5)
uERA5_ver_JJA_cli = uERA5_ver_JJA.copy()
uERA5_ver_MAM = ca.detrend_dim(uERA5_ver_MAM, "time", deg=1, demean=False)
uERA5_ver_JJA = ca.detrend_dim(uERA5_ver_JJA, "time", deg=1, demean=False)
uERA5_ver_SON = ca.detrend_dim(uERA5_ver_SON, "time", deg=1, demean=False)
uERA5_ver_DJF = ca.detrend_dim(uERA5_ver_DJF, "time", deg=1, demean=False)

vERA5_ver_MAM, vERA5_ver_JJA, vERA5_ver_SON, vERA5_ver_DJF = ca.mon_to_season(vERA5)
vERA5_ver_MAM = ca.detrend_dim(vERA5_ver_MAM, "time", deg=1, demean=False)
vERA5_ver_JJA = ca.detrend_dim(vERA5_ver_JJA, "time", deg=1, demean=False)
vERA5_ver_SON = ca.detrend_dim(vERA5_ver_SON, "time", deg=1, demean=False)
vERA5_ver_DJF = ca.detrend_dim(vERA5_ver_DJF, "time", deg=1, demean=False)

qERA5_ver_JJA = ca.p_time(qERA5, 6, 8, True)
qERA5_ver_JJA = ca.detrend_dim(qERA5_ver_JJA, "time", deg=1, demean=False)

spERA5_MAM, spERA5_JJA, spERA5_SON, spERA5_DJF = ca.mon_to_season(spERA5)
spERA5_MAM = ca.detrend_dim(spERA5_MAM, "time", deg=1, demean=False)
spERA5_JJA = ca.detrend_dim(spERA5_JJA, "time", deg=1, demean=False)
spERA5_SON = ca.detrend_dim(spERA5_SON, "time", deg=1, demean=False)
spERA5_DJF = ca.detrend_dim(spERA5_DJF, "time", deg=1, demean=False)

wERA5_MAM, wERA5_JJA, wERA5_SON, wERA5_DJF = ca.mon_to_season(wERA5)
wERA5_MAM = ca.detrend_dim(wERA5_MAM, "time", deg=1, demean=False)
wERA5_JJA = ca.detrend_dim(wERA5_JJA, "time", deg=1, demean=False)
wERA5_SON = ca.detrend_dim(wERA5_SON, "time", deg=1, demean=False)
wERA5_DJF = ca.detrend_dim(wERA5_DJF, "time", deg=1, demean=False)
sstHad_MAM, sstHad_JJA, sstHad_SON, sstHad_DJF = ca.mon_to_season(sstHad)
sstHad_MAM = ca.detrend_dim(sstHad_MAM, "time", deg=1, demean=False)
sstHad_JJA = ca.detrend_dim(sstHad_JJA, "time", deg=1, demean=False)
sstHad_SON = ca.detrend_dim(sstHad_SON, "time", deg=1, demean=False)
sstHad_DJF = ca.detrend_dim(sstHad_DJF, "time", deg=1, demean=False)
ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)
tERA5_ver_JJA = ca.detrend_dim(tERA5_ver_JJA, "time", deg=1, demean=False)
divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))
divERA5_ver_JJA = ca.detrend_dim(divERA5_ver_JJA, "time", deg=1, demean=False)

#   calculate the waver vapor vertical intergration
ptop = 100
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= ptop, drop=True) * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5level.coords["level"] = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= ptop, drop=True) * 100.0
ERA5level.coords["level"].attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, 10000)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
integration_top = 200
uq_sl_ERA5_JJA = uERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
vqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
uqERA5_ver_JJA = ca.detrend_dim(uqERA5_ver_JJA, "time", deg=1, demean=False)
vqERA5_ver_JJA = ca.detrend_dim(vqERA5_ver_JJA, "time", deg=1, demean=False)
# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)
uqvqdivERA5_ver_JJA = ca.detrend_dim(uqvqdivERA5_ver_JJA, "time", deg=1, demean=False)

slhf_ERA5 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/mslhf_mon_r144x72_197901-201412.nc")*0.408*1e-6
slhf_ERA5_JJA = ca.p_time(slhf_ERA5, 6, 8, True)

In [ ]:
ERA5dpncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/ERA5dp_calbyncl.nc")

hus_ERA5_prime_dp_ncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/husERA5_JJA_detrend_dp.nc")
hus_ERA5_prime_dy_ncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/husERA5_JJA_detrend_dy.nc")
hus_ERA5_prime_dx_ncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/husERA5_JJA_detrend_dx.nc")

hus_ERA5_bar_dp_ncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/husERA5_JJA_bar_dp.nc")
hus_ERA5_bar_dy_ncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/husERA5_JJA_bar_dy.nc")
hus_ERA5_bar_dx_ncl = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/husERA5_JJA_bar_dx.nc")


# 降水分区

In [13]:
lat = preGPCP_JJA.coords["lat"]
lon = preGPCP_JJA.coords["lon"]
# # 北部
# nIndia_N = 35.0
# nIndia_S = 20.0
# nIndia_W = 70.0
# nIndia_E = 85.0
# lat_nIndia_range = lat[(lat >= nIndia_S) & (lat <= nIndia_N)]
# lon_nIndia_range = lon[(lon >= nIndia_W) & (lon <= nIndia_E)]

# preGPCP_nIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
# prehis_nIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
# pressp245_p3_nIndia_JJA = ca.cal_area_weighted_mean(pressp245_p3_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
# pressp585_p3_nIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))

# # detrend
# preGPCP_nIndia_JJA_detrend = ca.detrend_dim(preGPCP_nIndia_JJA.copy(), "time", deg=1, demean=True)
# prehis_nIndia_JJA_detrend = ca.detrend_dim(prehis_nIndia_JJA.copy(), "time", deg=1, demean=True)
# pressp245_p3_nIndia_JJA_detrend = ca.detrend_dim(pressp245_p3_nIndia_JJA.copy(), "time", deg=1, demean=True)
# pressp585_p3_nIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_nIndia_JJA.copy(), "time", deg=1, demean=True)

# # standardize
# preGPCP_nIndia_JJA_nor = ca.standardize(preGPCP_nIndia_JJA_detrend)
# prehis_nIndia_JJA_nor = ca.standardize(prehis_nIndia_JJA_detrend)
# pressp245_p3_nIndia_JJA_nor = ca.standardize(pressp245_p3_nIndia_JJA_detrend)
# pressp585_p3_nIndia_JJA_nor = ca.standardize(pressp585_p3_nIndia_JJA_detrend)

# # 南部
# sIndia_N = 20.0
# sIndia_S = 5.0
# sIndia_W = 70.0
# sIndia_E = 85.0
# lat_sIndia_range = lat[(lat >= sIndia_S) & (lat <= sIndia_N)]
# lon_sIndia_range = lon[(lon >= sIndia_W) & (lon <= sIndia_E)]

# preGPCP_sIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
# prehis_sIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
# pressp245_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp245_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
# pressp585_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))

# # detrend
# preGPCP_sIndia_JJA_detrend = ca.detrend_dim(preGPCP_sIndia_JJA.copy(), "time", deg=1, demean=True)
# prehis_sIndia_JJA_detrend = ca.detrend_dim(prehis_sIndia_JJA.copy(), "time", deg=1, demean=True)
# pressp245_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp245_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)
# pressp585_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)

# # standardize
# preGPCP_sIndia_JJA_nor = ca.standardize(preGPCP_sIndia_JJA_detrend)
# prehis_sIndia_JJA_nor = ca.standardize(prehis_sIndia_JJA_detrend)
# pressp245_p3_sIndia_JJA_nor = ca.standardize(pressp245_p3_sIndia_JJA_detrend)
# pressp585_p3_sIndia_JJA_nor = ca.standardize(pressp585_p3_sIndia_JJA_detrend)

# 全印度
India_N = 35.0
India_S = 5.0
India_W = 70.0
India_E = 85.0
lat_India_range = lat[(lat >= India_S) & (lat <= India_N)]
lon_India_range = lon[(lon >= India_W) & (lon <= India_E)]

preGPCP_India_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_India_range, lon=lon_India_range))
# prehis_India_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range))
# pressp245_p3_India_JJA = ca.cal_area_weighted_mean(pressp245_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range))
# pressp585_p3_India_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range))

# detrend
preGPCP_India_JJA_detrend = ca.detrend_dim(preGPCP_India_JJA.copy(), "time", deg=1, demean=True)
# prehis_India_JJA_detrend = ca.detrend_dim(prehis_India_JJA.copy(), "time", deg=1, demean=True)
# pressp245_p3_India_JJA_detrend = ca.detrend_dim(pressp245_p3_India_JJA.copy(), "time", deg=1, demean=True)
# pressp585_p3_India_JJA_detrend = ca.detrend_dim(pressp585_p3_India_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_India_JJA_nor = ca.standardize(preGPCP_India_JJA_detrend)
# prehis_India_JJA_nor = ca.standardize(prehis_India_JJA_detrend)
# pressp245_p3_India_JJA_nor = ca.standardize(pressp245_p3_India_JJA_detrend)
# pressp585_p3_India_JJA_nor = ca.standardize(pressp585_p3_India_JJA_detrend)

# # 华北地区
# NC_N = 40.0
# NC_S = 32.5
# NC_W = 105.0
# NC_E = 120.0
# lat_NC_range = lat[(lat >= NC_S) & (lat <= NC_N)]
# lon_NC_range = lon[(lon >= NC_W) & (lon <= NC_E)]
# preGPCP_NC_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_NC_range, lon=lon_NC_range))
# preGPCP_NC_JJA = ca.detrend_dim(preGPCP_NC_JJA, "time", deg=1, demean=True)
# # standardize
# preGPCP_NC_JJA_nor = ca.standardize(preGPCP_NC_JJA)

# prehis_NC_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_NC_range, lon=lon_NC_range))
# prehis_NC_JJA = ca.detrend_dim(prehis_NC_JJA, "time", deg=1, demean=True)
# # standardize
# prehis_NC_JJA_nor = ca.standardize(prehis_NC_JJA)

# # 长江中下游地区
# YZR_N = 32.5
# YZR_S = 27.5
# YZR_W = 105.0
# YZR_E = 120.0
# lat_YZR_range = lat[(lat >= YZR_S) & (lat <= YZR_N)]
# lon_YZR_range = lon[(lon >= YZR_W) & (lon <= YZR_E)]
# preGPCP_YZR_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_YZR_range, lon=lon_YZR_range))
# preGPCP_YZR_JJA = ca.detrend_dim(preGPCP_YZR_JJA, "time", deg=1, demean=True)
# # standardize
# preGPCP_YZR_JJA_nor = ca.standardize(preGPCP_YZR_JJA)

# prehis_YZR_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_YZR_range, lon=lon_YZR_range))
# prehis_YZR_JJA = ca.detrend_dim(prehis_YZR_JJA, "time", deg=1, demean=True)
# # standardize
# prehis_YZR_JJA_nor = ca.standardize(prehis_YZR_JJA)

# # 日本南部
# SJ_N = 35
# SJ_S = 30
# SJ_W = 130.0
# SJ_E = 137.5
# lat_SJ_range = lat[(lat >= SJ_S) & (lat <= SJ_N)]
# lon_SJ_range = lon[(lon >= SJ_W) & (lon <= SJ_E)]
# preGPCP_SJ_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_SJ_range, lon=lon_SJ_range))
# preGPCP_SJ_JJA = ca.detrend_dim(preGPCP_SJ_JJA, "time", deg=1, demean=True)
# # standardize
# preGPCP_SJ_JJA_nor = ca.standardize(preGPCP_SJ_JJA)

# prehis_SJ_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_SJ_range, lon=lon_SJ_range))
# prehis_SJ_JJA = ca.detrend_dim(prehis_SJ_JJA, "time", deg=1, demean=True)
# # standardize
# prehis_SJ_JJA_nor = ca.standardize(prehis_SJ_JJA)


# 计算气候态平均和距平 

In [18]:
#观测ERA5和GPCP结果
pre_ERA5_bar = ca.p_time(preGPCP, 6, 8, True).mean(dim="time")
pre_ERA5_prime = ca.p_time(preGPCP, 6, 8, True)-pre_ERA5_bar

u_ERA5_bar = ca.p_time(uERA5, 6, 8, True).mean(dim="time")
u_ERA5_prime = ca.p_time(uERA5, 6, 8, True)-u_ERA5_bar

v_ERA5_bar = ca.p_time(vERA5, 6, 8, True).mean(dim="time")
v_ERA5_prime = ca.p_time(vERA5, 6, 8, True)-v_ERA5_bar

hus_ERA5_bar = ca.p_time(qERA5, 6, 8, True).mean(dim="time")
hus_ERA5_prime = ca.p_time(qERA5, 6, 8, True)-hus_ERA5_bar

wap_ERA5_bar = ca.p_time(wERA5, 6, 8, True).mean(dim="time")
wap_ERA5_prime = ca.p_time(wERA5, 6, 8, True)-wap_ERA5_bar

hfls_ERA5_prime = slhf_ERA5_JJA - slhf_ERA5_JJA.mean(dim="time")

In [ ]:
pre_ERA5_prime.mean(dim="time")

In [ ]:
level = qERA5_ver_JJA.level
sel_level = level[level>=100.0]

def recoordinate(var, level,  lat, lon):
  var.coords["level"] = level
  var.coords["lat"] = lat
  var.coords["lon"] = lon
  return(var)

def create_dpdydx(models, sel_level, lat, lon):
  tmp_dp_array, tmp_dx_array, tmp_dy_array = np.zeros((len(models), len(sel_level), len(lat), len(lon))), np.zeros((len(models), len(sel_level), len(lat), len(lon))), np.zeros((len(models), len(sel_level), len(lat), len(lon)))
  tmp_dp = xr.DataArray(
  data=tmp_dp_array, 
  dims=["models", "level", "lat", "lon"],
  coords=dict(
    models=models.data,
    level=sel_level.data,
    lat=lat.data,
    lon=lon.data
    )
  )

  tmp_dy = xr.DataArray(
    data=tmp_dy_array, 
    dims=["models", "level", "lat", "lon"],
    coords=dict(
      models=models.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )

  tmp_dx = xr.DataArray(
    data=tmp_dx_array, 
    dims=["models", "level", "lat", "lon"],
    coords=dict(
      models=models.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )
  return(tmp_dp, tmp_dy, tmp_dx)

def create_ERA5_dpdydx(time, sel_level, lat, lon):
  tmp_dp_array, tmp_dx_array, tmp_dy_array = np.zeros((len(time), len(sel_level), len(lat), len(lon))), np.zeros((len(time), len(sel_level), len(lat), len(lon))), np.zeros((len(time), len(sel_level), len(lat), len(lon)))
  tmp_dp = xr.DataArray(
  data=tmp_dp_array, 
  dims=["time", "level", "lat", "lon"],
  coords=dict(
    time=time.data,
    level=sel_level.data,
    lat=lat.data,
    lon=lon.data
    )
  )

  tmp_dy = xr.DataArray(
    data=tmp_dy_array, 
    dims=["time", "level", "lat", "lon"],
    coords=dict(
      time=time.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )

  tmp_dx = xr.DataArray(
    data=tmp_dx_array, 
    dims=["time", "level", "lat", "lon"],
    coords=dict(
      time=time.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )
  return(tmp_dp, tmp_dy, tmp_dx)

def create_time_dpdydx(time, models, sel_level, lat, lon):
  tmp_dp_array, tmp_dx_array, tmp_dy_array = np.zeros((len(time), len(models), len(sel_level), len(lat), len(lon))), np.zeros((len(time), len(models), len(sel_level), len(lat), len(lon))), np.zeros((len(time), len(models), len(sel_level), len(lat), len(lon)))
  tmp_dp = xr.DataArray(
  data=tmp_dp_array, 
  dims=["time", "models", "level", "lat", "lon"],
  coords=dict(
    time=time.data,
    models=models.data,
    level=sel_level.data,
    lat=lat.data,
    lon=lon.data
    )
  )

  tmp_dy = xr.DataArray(
    data=tmp_dy_array, 
    dims=["time", "models", "level", "lat", "lon"],
    coords=dict(
      time=time.data,
      models=models.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )

  tmp_dx = xr.DataArray(
    data=tmp_dx_array, 
    dims=["time", "models", "level", "lat", "lon"],
    coords=dict(
      time=time.data,
      models=models.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )
  return(tmp_dp, tmp_dy, tmp_dx)



# 计算水汽的偏微分

In [ ]:
# IndR_ERA5_hus_slope.attrs["units"] = "1"
time = hus_ERA5_prime.coords["time"]
hus_ERA5_prime_dp, hus_ERA5_prime_dy, hus_ERA5_prime_dx = create_ERA5_dpdydx(time, sel_level*100, lat, lon)
hus_ERA5_prime.coords["level"] = uERA5_ver_JJA.level*100
hus_ERA5_prime.coords["level"].attrs["units"] = "Pa"

for num_t, t in enumerate(time):
  hus_ERA5_prime_dp[num_t], hus_ERA5_prime_dy[num_t], hus_ERA5_prime_dx[num_t] = mpcalc.gradient(hus_ERA5_prime.sel(time=t, level=sel_level*100))

hus_ERA5_bar.coords["level"] = uERA5_ver_JJA.level*100
hus_ERA5_bar.coords["level"].attrs['units'] = "Pa"
hus_ERA5_bar_dp, hus_ERA5_bar_dy, hus_ERA5_bar_dx = mpcalc.gradient(hus_ERA5_bar.sel(level=sel_level*100))

# dp项除以100是因为做垂直方向的微分时，以hPa为dp进行了计算
hus_ERA5_prime_dp = hus_ERA5_prime_dp.metpy.dequantify()
hus_ERA5_prime_dx = hus_ERA5_prime_dx.metpy.dequantify()
hus_ERA5_prime_dy = hus_ERA5_prime_dy.metpy.dequantify()

hus_ERA5_bar_dp = hus_ERA5_bar_dp.metpy.dequantify()
hus_ERA5_bar_dx = hus_ERA5_bar_dx.metpy.dequantify()
hus_ERA5_bar_dy = hus_ERA5_bar_dy.metpy.dequantify()

hus_ERA5_prime.coords["level"] = level
hus_ERA5_bar.coords["level"] = level
hus_ERA5_prime_dp.coords["level"] = sel_level
hus_ERA5_prime_dx.coords["level"] = sel_level
hus_ERA5_prime_dy.coords["level"] = sel_level
hus_ERA5_bar_dp.coords["level"] = sel_level
hus_ERA5_bar_dx.coords["level"] = sel_level
hus_ERA5_bar_dy.coords["level"] = sel_level

In [ ]:
def vert_intergrate(da_sl, CMIP6dpg):
  res_vert = (da_sl * CMIP6dpg.data).sum(dim="level")
  return(res_vert)

In [ ]:
tmp1, tmp2 = xr.broadcast(u_ERA5_bar.sel(level=sel_level), hus_ERA5_prime_dx)
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_1_1 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
# print(tmp1, tmp2)
del(tmp1, tmp2)

tmp1, tmp2 = xr.broadcast(v_ERA5_bar.sel(level=sel_level), hus_ERA5_prime_dy)
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_1_2 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
# print(tmp1, tmp2)
del(tmp1, tmp2)

term_ERA5_1 = term_ERA5_1_1 + term_ERA5_1_2

tmp1, tmp2 = xr.broadcast(u_ERA5_prime.sel(level=sel_level), hus_ERA5_bar_dx)
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_2_1 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
# print(tmp1, tmp2)
del(tmp1, tmp2)

tmp1, tmp2 = xr.broadcast(v_ERA5_prime.sel(level=sel_level), hus_ERA5_bar_dy)
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_2_2 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
# print(tmp1, tmp2)
del(tmp1, tmp2)
term_ERA5_2 = term_ERA5_2_1+term_ERA5_2_2

tmp1, tmp2 = xr.broadcast(wap_ERA5_bar.sel(level=sel_level), hus_ERA5_prime_dp)
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_3 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
# print(tmp1, tmp2)
del(tmp1, tmp2)

tmp1, tmp2 = xr.broadcast(wap_ERA5_prime.sel(level=sel_level), hus_ERA5_bar_dp)
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_4 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
# print(tmp1, tmp2)
del(tmp1, tmp2)

pre_ERA5_prime.coords["time"] = preGPCP_India_JJA_nor.time
hfls_ERA5_prime.coords["time"] = preGPCP_India_JJA_nor.time
term_ERA5_1.coords["time"] = preGPCP_India_JJA_nor.time
term_ERA5_2.coords["time"] = preGPCP_India_JJA_nor.time
term_ERA5_3.coords["time"] = preGPCP_India_JJA_nor.time
term_ERA5_4.coords["time"] = preGPCP_India_JJA_nor.time


term_ERA5_NLRes = pre_ERA5_prime+hfls_ERA5_prime+term_ERA5_1+term_ERA5_2+term_ERA5_3+term_ERA5_4

In [ ]:
NC_N = 40.0
NC_S = 32.5
NC_W = 105.0
NC_E = 120.0
lat = preGPCP_JJA.coords["lat"]
lon = preGPCP_JJA.coords["lon"]
lat_NC_range = lat[(lat >= NC_S) & (lat <= NC_N)]
lon_NC_range = lon[(lon >= NC_W) & (lon <= NC_E)]
print(pre_ERA5_prime[0].sel(lat=lat_NC_range, lon=lon_NC_range).mean(dim=["lat", "lon"]).data)

print((-hfls_ERA5_prime[0].sel(lat=lat_NC_range, lon=lon_NC_range)).mean(dim=["lat", "lon"]).data)

print((-term_ERA5_1[0].sel(lat=lat_NC_range, lon=lon_NC_range)).mean(dim=["lat", "lon"]).data)

print((-term_ERA5_2[0].sel(lat=lat_NC_range, lon=lon_NC_range)).mean(dim=["lat", "lon"]).data)

print((-term_ERA5_3[0].sel(lat=lat_NC_range, lon=lon_NC_range)).mean(dim=["lat", "lon"]).data)

print((-term_ERA5_4[0].sel(lat=lat_NC_range, lon=lon_NC_range)).mean(dim=["lat", "lon"]).data)

In [ ]:
print((term_ERA5_NLRes[0].sel(lat=lat_NC_range, lon=lon_NC_range)).mean(dim=["lat", "lon"]).data)

In [ ]:
(
    term_ERA5_1_slope,
    term_ERA5_1_intercept,
    term_ERA5_1_rvalue,
    term_ERA5_1_pvalue,
    term_ERA5_1_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_1)

term_ERA5_1_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], term_ERA5_1_slope.data),
        intercept=(["lat", "lon"], term_ERA5_1_intercept.data),
        rvalue=(["lat", "lon"], term_ERA5_1_rvalue.data),
        pvalue=(["lat", "lon"], term_ERA5_1_pvalue.data),
        hypothesis=(["lat", "lon"], term_ERA5_1_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="term_ERA5_1 fields regress"),
)

ca.save(term_ERA5_1_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ERA5_1_regress.nc")

(
    term_ERA5_2_slope,
    term_ERA5_2_intercept,
    term_ERA5_2_rvalue,
    term_ERA5_2_pvalue,
    term_ERA5_2_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_2)

term_ERA5_2_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], term_ERA5_2_slope.data),
        intercept=(["lat", "lon"], term_ERA5_2_intercept.data),
        rvalue=(["lat", "lon"], term_ERA5_2_rvalue.data),
        pvalue=(["lat", "lon"], term_ERA5_2_pvalue.data),
        hypothesis=(["lat", "lon"], term_ERA5_2_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="term_ERA5_2 fields regress"),
)

ca.save(term_ERA5_2_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ERA5_2_regress.nc")

(
    term_ERA5_3_slope,
    term_ERA5_3_intercept,
    term_ERA5_3_rvalue,
    term_ERA5_3_pvalue,
    term_ERA5_3_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_3)

term_ERA5_3_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], term_ERA5_3_slope.data),
        intercept=(["lat", "lon"], term_ERA5_3_intercept.data),
        rvalue=(["lat", "lon"], term_ERA5_3_rvalue.data),
        pvalue=(["lat", "lon"], term_ERA5_3_pvalue.data),
        hypothesis=(["lat", "lon"], term_ERA5_3_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="term_ERA5_3 fields regress"),
)

ca.save(term_ERA5_3_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ERA5_3_regress.nc")

(
    term_ERA5_4_slope,
    term_ERA5_4_intercept,
    term_ERA5_4_rvalue,
    term_ERA5_4_pvalue,
    term_ERA5_4_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_4)

term_ERA5_4_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], term_ERA5_4_slope.data),
        intercept=(["lat", "lon"], term_ERA5_4_intercept.data),
        rvalue=(["lat", "lon"], term_ERA5_4_rvalue.data),
        pvalue=(["lat", "lon"], term_ERA5_4_pvalue.data),
        hypothesis=(["lat", "lon"], term_ERA5_4_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="term_ERA5_4 fields regress"),
)

ca.save(term_ERA5_4_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ERA5_4_regress.nc")

(
    term_ERA5_NLRes_slope,
    term_ERA5_NLRes_intercept,
    term_ERA5_NLRes_rvalue,
    term_ERA5_NLRes_pvalue,
    term_ERA5_NLRes_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_NLRes)

term_ERA5_NLRes_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], term_ERA5_NLRes_slope.data),
        intercept=(["lat", "lon"], term_ERA5_NLRes_intercept.data),
        rvalue=(["lat", "lon"], term_ERA5_NLRes_rvalue.data),
        pvalue=(["lat", "lon"], term_ERA5_NLRes_pvalue.data),
        hypothesis=(["lat", "lon"], term_ERA5_NLRes_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="term_ERA5_NLRes fields regress"),
)

ca.save(term_ERA5_NLRes_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ERA5_NLRes_regress.nc")

In [ ]:
NC_N = 40.0
NC_S = 32.5
NC_W = 105.0
NC_E = 120.0
lat_NC_range = lat[(lat >= NC_S) & (lat <= NC_N)]
lon_NC_range = lon[(lon >= NC_W) & (lon <= NC_E)]
obs_NC_qbudget = np.zeros(7)

obs_NC_qbudget[0] = ca.cal_area_weighted_mean(ca.dim_linregress(preGPCP_India_JJA_nor, preGPCP_JJA)[0].sel(lat=lat_NC_range, lon=lon_NC_range)).data
obs_NC_qbudget[1] = ca.cal_area_weighted_mean(ca.dim_linregress(preGPCP_India_JJA_nor, hfls_ERA5_prime)[0].sel(lat=lat_NC_range, lon=lon_NC_range)).data

obs_NC_qbudget[2] = ca.cal_area_weighted_mean(-term_ERA5_1_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data
obs_NC_qbudget[3] = ca.cal_area_weighted_mean(-term_ERA5_2_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data
obs_NC_qbudget[4] = ca.cal_area_weighted_mean(-term_ERA5_3_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data
obs_NC_qbudget[5] = ca.cal_area_weighted_mean(-term_ERA5_4_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data
obs_NC_qbudget[6] = ca.cal_area_weighted_mean(term_ERA5_NLRes_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data
obs_NC_qbudget

In [ ]:
# 计算散度

uvdiv_bar  = mpcalc.divergence(u_ERA5_bar, v_ERA5_bar).metpy.dequantify()

uvdiv_prime  = mpcalc.divergence(u_ERA5_prime, v_ERA5_prime).metpy.dequantify()

omega_bar_dp = (mpcalc.gradient(wap_ERA5_bar)[0]).metpy.dequantify()/100

omega_prime_dp = (mpcalc.gradient(wap_ERA5_prime)[0]).metpy.dequantify()/100

In [ ]:
tmp1, tmp2 = xr.broadcast(uvdiv_bar.sel(level=sel_level), hus_ERA5_prime.sel(level=sel_level))
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_test_1 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
del(tmp1, tmp2)

tmp1, tmp2 = xr.broadcast(uvdiv_prime.sel(level=sel_level), hus_ERA5_bar.sel(level=sel_level))
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_test_2 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
del(tmp1, tmp2)

In [ ]:
(
    term_ERA5_test_1_slope,
    term_ERA5_test_1_intercept,
    term_ERA5_test_1_rvalue,
    term_ERA5_test_1_pvalue,
    term_ERA5_test_1_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_test_1)


(
    term_ERA5_test_2_slope,
    term_ERA5_test_2_intercept,
    term_ERA5_test_2_rvalue,
    term_ERA5_test_2_pvalue,
    term_ERA5_test_2_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_test_2)

In [ ]:
ca.cal_area_weighted_mean(-term_ERA5_test_1_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data

In [ ]:
ca.cal_area_weighted_mean(-term_ERA5_test_2_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data

In [ ]:
tmp1, tmp2 = xr.broadcast(omega_bar_dp.sel(level=sel_level), hus_ERA5_prime.sel(level=sel_level))
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_test_3 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
del(tmp1, tmp2)

tmp1, tmp2 = xr.broadcast(omega_prime_dp.sel(level=sel_level), hus_ERA5_bar.sel(level=sel_level))
tmp1 = tmp1.transpose("time", ...)
tmp2 = tmp2.transpose("time", ...)
term_ERA5_test_4 = vert_intergrate(tmp1*tmp2, CMIP6dpg=ERA5dpncl/g)
del(tmp1, tmp2)

In [ ]:
(
    term_ERA5_test_3_slope,
    term_ERA5_test_3_intercept,
    term_ERA5_test_3_rvalue,
    term_ERA5_test_3_pvalue,
    term_ERA5_test_3_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_test_3)


(
    term_ERA5_test_4_slope,
    term_ERA5_test_4_intercept,
    term_ERA5_test_4_rvalue,
    term_ERA5_test_4_pvalue,
    term_ERA5_test_4_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, term_ERA5_test_4)

In [ ]:
ca.cal_area_weighted_mean(-term_ERA5_test_3_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data

In [ ]:
ca.cal_area_weighted_mean(-term_ERA5_test_4_slope.sel(lat=lat_NC_range, lon=lon_NC_range)).data

In [56]:
zzy_cal = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/ERA5_ncl_qbudget_zzy_1x1.nc")
cal_pre = zzy_cal["pre"]
cal_E = zzy_cal["E"]
cal_T1 = zzy_cal["T1"]
cal_T2 = zzy_cal["T2"]
cal_T3 = zzy_cal["T3"]
cal_T4 = zzy_cal["T4"]
cal_T5 = zzy_cal["T5"]

In [57]:
for_latlon = xr.open_dataarray("/home/yangsong3/Extension/ERA5/ERA5-monthly/pressure/uwind_mon_1x1_1950_202106.nc")
lat = for_latlon["latitude"]
lon = for_latlon["longitude"]

In [58]:
def cal_area_weighted_mean(da):
    """计算数据的区域（纬度加权）平均

    Args:
        da (dataarray): 原数据

    Returns:
        dataarray: 区域平均结果
    """    
    weights = np.cos(np.deg2rad(da.latitude))
    weights.name = "weights"
    da_mean = da.weighted(weights).mean(("latitude"), skipna=True)
    da_mean = da_mean.mean(dim="longitude", skipna=True)
    return da_mean

In [62]:
NC_N = 40.0
NC_S = 32.5
NC_W = 105.0
NC_E = 120.0
lat_NC_range = lat[(lat >= NC_S) & (lat <= NC_N)]
lon_NC_range = lon[(lon >= NC_W) & (lon <= NC_E)]

print(cal_area_weighted_mean(cal_pre[0].loc[40:44, 79:93]).data)

print(cal_area_weighted_mean(cal_E[0].loc[40:44, 79:93]).data)

print(cal_area_weighted_mean(cal_T1[0].loc[40:44, 79:93]).data)

print(cal_area_weighted_mean(cal_T2[0].loc[40:44, 79:93]).data)

print(cal_area_weighted_mean(cal_T3[0].loc[40:44, 79:93]).data)

print(cal_area_weighted_mean(cal_T4[0].loc[40:44, 79:93]).data)

print(cal_area_weighted_mean(cal_T5[0].loc[40:44, 79:93]).data)

nan
nan
nan
nan
nan
nan
nan


In [60]:
preGPCP_India_JJA_nor.coords["time"] = cal_pre.time
(
    term_ERA5_nclzzy_pre_slope,
    term_ERA5_nclzzy_pre_intercept,
    term_ERA5_nclzzy_pre_rvalue,
    term_ERA5_nclzzy_pre_pvalue,
    term_ERA5_nclzzy_pre_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_pre)

(
    term_ERA5_nclzzy_E_slope,
    term_ERA5_nclzzy_E_intercept,
    term_ERA5_nclzzy_E_rvalue,
    term_ERA5_nclzzy_E_pvalue,
    term_ERA5_nclzzy_E_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_E)

(
    term_ERA5_nclzzy_T1_slope,
    term_ERA5_nclzzy_T1_intercept,
    term_ERA5_nclzzy_T1_rvalue,
    term_ERA5_nclzzy_T1_pvalue,
    term_ERA5_nclzzy_T1_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_T1)

(
    term_ERA5_nclzzy_T2_slope,
    term_ERA5_nclzzy_T2_intercept,
    term_ERA5_nclzzy_T2_rvalue,
    term_ERA5_nclzzy_T2_pvalue,
    term_ERA5_nclzzy_T2_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_T2)

(
    term_ERA5_nclzzy_T3_slope,
    term_ERA5_nclzzy_T3_intercept,
    term_ERA5_nclzzy_T3_rvalue,
    term_ERA5_nclzzy_T3_pvalue,
    term_ERA5_nclzzy_T3_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_T3)

(
    term_ERA5_nclzzy_T4_slope,
    term_ERA5_nclzzy_T4_intercept,
    term_ERA5_nclzzy_T4_rvalue,
    term_ERA5_nclzzy_T4_pvalue,
    term_ERA5_nclzzy_T4_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_T4)


(
    term_ERA5_nclzzy_T5_slope,
    term_ERA5_nclzzy_T5_intercept,
    term_ERA5_nclzzy_T5_rvalue,
    term_ERA5_nclzzy_T5_pvalue,
    term_ERA5_nclzzy_T5_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, cal_T5)

In [67]:
ERA5pre = xr.open_dataset("/home/yangsong3/Extension/ERA5/ERA5-monthly/surface/prect_mon_1x1_1950_202106.nc")
ERA5_pre_JJA = ERA5pre["tp"]
ERA5_pre_JJA = ERA5_pre_JJA.sel(time=(ERA5_pre_JJA.time.dt.year>=1979) & (ERA5_pre_JJA.time.dt.year<=2014))
ERA5_pre_JJA = ca.p_time(ERA5_pre_JJA, 6, 8)

In [73]:
ERA5_uqvq = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/ERA5_ncl_uqvq_zzy_1x1.nc")
uq_test = ERA5_uqvq["uq_vert"]
vq_test = ERA5_uqvq["vq_vert"]

uqvqdiv = mpcalc.divergence(uq_test, vq_test).metpy.dequantify()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/pint/quantity.py:1313: RuntimeWarning: invalid value encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [75]:
stats.linregress(cal_area_weighted_mean(ERA5_pre_JJA.sel(latitude=lat_NC_range, longitude=lon_NC_range)), cal_area_weighted_mean(uqvqdiv.loc[:,40:32.5, 105:120]))

LinregressResult(slope=-11.189837227713728, intercept=0.024436641433472954, rvalue=-0.702158578121713, pvalue=1.81825689898035e-06, stderr=1.945994739222998, intercept_stderr=0.008336372649421737)

In [50]:
print(term_ERA5_nclzzy_pre_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

print(term_ERA5_nclzzy_E_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

print(term_ERA5_nclzzy_T1_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

print(term_ERA5_nclzzy_T2_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

print(term_ERA5_nclzzy_T3_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

print(term_ERA5_nclzzy_T4_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

print(term_ERA5_nclzzy_T5_slope.sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data)

0.19689981165399806
-0.00036795961563214005
0.01630559745030824
0.17121684516431784
-0.07023896878330815
0.2576180282953653
0.0025761802751153207


In [21]:

preGPCP_India_JJA_nor.coords["time"] = preGPCP_JJA.time
ca.cal_area_weighted_mean(ca.dim_linregress(preGPCP_India_JJA_nor, preGPCP_JJA)[0].loc[32.5:40,105:120]).data

array(2.26064634e-06)

In [ ]:
res = cal_pre[0].sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"]).data/86400-(cal_E[0].sel(latitude=lat_NC_range, longitude=lon_NC_range)).mean(dim=["latitude", "longitude"]).data/86400

In [10]:
preERA5 = xr.open_dataarray("/home/yangsong3/Extension/ERA5/ERA5-monthly/surface/prect_mon_1x1_1950_202106.nc")

preERA5_JJA = ca.p_month(preERA5.sel(time=((preERA5.time.dt.year>=1979)&(preERA5.time.dt.year<=2014))), 6, 8)



# preERA5_JJA = ca.p_time(preERA5.sel(time=((preERA5.time.dt.year>=1979)&(preERA5.time.dt.year<=2014))), 6, 8)
# (preERA5_JJA-preERA5_JJA.mean(dim="time"))[0].sel(latitude=lat_NC_range, longitude=lon_NC_range).mean(dim=["latitude", "longitude"])*1000/86400

zzy

In [42]:
test_cal = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/ERA5_ncl_qbudget_test_1x1.nc")
cal_test_pre = test_cal["pre"]
cal_test_E = test_cal["E"]
cal_test_T1 = test_cal["T1"]
cal_test_T2 = test_cal["T2"]
cal_test_T3 = test_cal["T3"]
cal_test_T4 = test_cal["T4"]


In [49]:
NC_N = 40.0
NC_S = 32.5
NC_W = 105.0
NC_E = 120.0
lat_NC_range = lat[(lat >= NC_S) & (lat <= NC_N)]
lon_NC_range = lon[(lon >= NC_W) & (lon <= NC_E)]

print(cal_area_weighted_mean(cal_test_pre.sel(latitude=lat_NC_range, longitude=lon_NC_range)).data)

print(cal_area_weighted_mean(cal_test_E.sel(latitude=lat_NC_range, longitude=lon_NC_range)).data)

print(cal_area_weighted_mean(cal_test_T1.sel(latitude=lat_NC_range, longitude=lon_NC_range)).data)

print(cal_area_weighted_mean(cal_test_T2.sel(latitude=lat_NC_range, longitude=lon_NC_range)).data)

print(cal_area_weighted_mean(cal_test_T3.sel(latitude=lat_NC_range, longitude=lon_NC_range)).data)

print(cal_area_weighted_mean(cal_test_T4.sel(latitude=lat_NC_range, longitude=lon_NC_range)).data)



-0.012849642
-0.020011118
0.01801371
-0.08497673
-0.20174782
0.10357812
